In [2]:
import sys
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

sys.path.append('../')

from utils.io import recursive_scan2df
from utils import endoscopy

# Real Setup Endoscopic View Segmentation
Extract endoscopic circle from real setup images

In [3]:
folder = 'img/21_04_02_calib'
df = recursive_scan2df('{}'.format(folder), '.npy')

# sort
df['index'] = df.file.apply(lambda x: int(x.replace('.', '_').split('_')[1]))
df.index = df.index.astype(float)
df = df.sort_values(by=['index'])

# bilateral filter on rgb
# doesnt matter if hsv or rgb -> hsv more intuitive though
# radius: gradient over search
# average area -> if mean > th, valid circle found, else previous or uninitialized

crp_count = 0
for _, row in df.iterrows():
    img = np.load(os.path.join(folder, row.folder, row.file))

    # mask
    mask_bil_hsv = endoscopy.bilateralSegmentation(img.astype(np.uint8), th=0.05, sigmaColor=75, sigmaSpace=75)

    # circle
    hsv_center, hsv_radius = endoscopy.boundaryCircle(mask_bil_hsv)
    top_left, shape = endoscopy.boundaryRectangle(mask_bil_hsv)

    if hsv_radius is None:
        continue

    # confidence measure for illumination
    level = endoscopy.illuminationLevel(mask_bil_hsv, hsv_center, hsv_radius)

    # crop
    if level > 0.98:
        inner_top_left, inner_shape = endoscopy.maxRectangleInCircle(img.shape, hsv_center, hsv_radius)
        inner_top_left, inner_shape = inner_top_left.astype(np.int), tuple(map(int, inner_shape))
        crp = endoscopy.crop(img, inner_top_left, inner_shape)
        crp_count += 1
        np.save('img/crop_{}.npy'.format(crp_count), crp)


    # int
    hsv_center, hsv_radius = hsv_center.astype(np.int), np.int(hsv_radius)
    top_left, shape = top_left.astype(np.int), tuple(map(int, shape))

    # bgr
    mask_bil_hsv = cv2.cvtColor(mask_bil_hsv, cv2.COLOR_GRAY2BGR)

    # draw
    cv2.circle(img, (hsv_center[1], hsv_center[0]), hsv_radius, (255, 255, 0), 2)
    cv2.circle(img, (hsv_center[1], hsv_center[0]), 2, (255, 255, 0), -1)

    cv2.rectangle(img, (top_left[1], top_left[0]), (top_left[1] + shape[1], top_left[0] + shape[0]), (255, 0, 255), 2)

    cv2.imshow('img', img)
    cv2.imshow('mask_bil_hsv', mask_bil_hsv)
    cv2.waitKey()
cv2.destroyAllWindows()

# Real Setup Homography Regression
Regress real setup homography from calibration pattern

In [1]:
import sys
import cv2
import os
import numpy as np

sys.path.append('../')

from utils.io import scan2df

# homography estimation
folder = 'img'
crp_df = scan2df(folder, '.npy')

for idx in range(len(crp_df) - 1):

    crp_0 = np.load(os.path.join(crp_df.iloc[idx].folder, crp_df.iloc[idx].file))
    crp_1 = np.load(os.path.join(crp_df.iloc[idx+1].folder, crp_df.iloc[idx+1].file))

    # homography estimation
    patternSize = (4, 11)

    # convert to grayscale
    crp_0 = cv2.cvtColor(crp_0.astype(np.float32), cv2.COLOR_BGR2GRAY).astype(np.uint8)
    crp_1 = cv2.cvtColor(crp_1.astype(np.float32), cv2.COLOR_BGR2GRAY).astype(np.uint8)

    # find points
    found0, pts0 = cv2.findCirclesGrid(crp_0, patternSize=patternSize, flags=cv2.CALIB_CB_ASYMMETRIC_GRID)
    found1, pts1 = cv2.findCirclesGrid(crp_1, patternSize=patternSize, flags=cv2.CALIB_CB_ASYMMETRIC_GRID)

    if found0 and found1:
        G, _ = cv2.findHomography(pts0, pts1, cv2.RANSAC)

        wrp = cv2.warpPerspective(crp_0, G, crp_1.shape[-2:][::-1])

        cv2.imshow('crp_0', crp_0)
        cv2.imshow('crp_1', crp_1)
        cv2.imshow('wrp', wrp)
        cv2.imshow('error', np.abs(wrp - crp_1))
        cv2.waitKey()
    else:
        print('Did not find points.')

cv2.destroyAllWindows()

# Real Setup Homography Regression
Extract homography from real setup using trained model and classical feature-based method

In [1]:
import sys
import os
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from kornia import image_to_tensor, tensor_to_image, warp_perspective

sys.path.append('../')

from utils.viz import yt_alpha_blend
from utils.processing import FeatureHomographyEstimation, four_point_homography_to_matrix, image_edges
from utils.io import recursive_scan2df, load_yaml
from utils import endoscopy
from lightning_modules import DeepImageHomographyEstimationModuleBackbone

# load network
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone/version_10'
# prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone_optimize_network/version_0'
configs = load_yaml(os.path.join(prefix, 'config.yml'))
model_input_shape = configs['model']['shape']
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=53-step=47573.ckpt'), shape=model_input_shape)
# model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=99-step=22099.ckpt'), shape=model_input_shape)

print('Model input shape: {}'.format(model_input_shape))

device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

model.to(device)
model = model.eval()
model.freeze()


# classical esimation
with_feature_estimation = False
fd = cv2.xfeatures2d.SIFT_create()
fh = FeatureHomographyEstimation(fd)

# image buffer
buffer = []

# load data
folder = 'img/21_04_02_abdomen'
df = recursive_scan2df('{}'.format(folder), '.npy')

# sort
df['index'] = df.file.apply(lambda x: int(x.replace('.', '_').split('_')[1]))
df.index = df.index.astype(float)
df = df.sort_values(by=['index'])

nth = 2
for _, row in df.iloc[::nth].iterrows():
    img = np.load(os.path.join(folder, row.folder, row.file))
    marked_img = img.copy()

    mask = endoscopy.bilateralSegmentation(img.astype(np.uint8), th=0.05)
    center, radius = endoscopy.boundaryCircle(mask)
    if radius is None:
        continue

    safety_margin = 10
    if radius > safety_margin:
        radius -= safety_margin  # safety margin
    top_left, shape = endoscopy.maxRectangleInCircle(mask.shape, center, radius)

    center, radius = center.astype(np.int), int(radius)
    top_left, shape = top_left.astype(np.int), tuple(map(int, shape))

    cv2.circle(marked_img, (center[1], center[0]), radius, (255, 255, 0), 2)
    cv2.rectangle(marked_img, (top_left[1], top_left[0]), (top_left[1]+shape[1], top_left[0]+shape[0]), (255, 0, 255), 2)

    # regress
    crp = endoscopy.crop(img, top_left, shape)
    if crp.shape[0] is 0:
        continue

    crp = cv2.resize(crp, (model_input_shape[2], model_input_shape[1]))

    buffer.append(crp)
    if len(buffer) > 2:
        buffer.pop(0)
    else:
        continue

    # classical feature prediction
    img, wrp = buffer[0], buffer[1]
    H_ft, duv_ft = fh(img.astype(np.uint8), wrp.astype(np.uint8))
    wrp_ft_pred = cv2.warpPerspective(img, H_ft, (img.shape[1], img.shape[0]))
    blend_ft = yt_alpha_blend(wrp, wrp_ft_pred)
    cv2.imshow('Classical: SIFT', blend_ft.astype(np.uint8))
    # blend_raw = yt_alpha_blend(wrp, img)
    # cv2.imshow('Raw', blend_raw.astype(np.uint8))

    img, wrp = image_to_tensor(img, keepdim=False).to(device, torch.float)/255, image_to_tensor(wrp, keepdim=False).to(device, torch.float)/255
    duv_deep = model(img, wrp)
    uv_deep = image_edges(img)
    H_deep = four_point_homography_to_matrix(uv_deep, duv_deep)
    wrp_deep_pred = warp_perspective(img, torch.inverse(H_deep), img.shape[-2:])
    blend_deep = yt_alpha_blend(wrp, wrp_deep_pred)
    blend_deep = tensor_to_image(blend_deep)
    cv2.imshow('Deep: Resnet34', blend_deep)

    cv2.imshow('marked_img', marked_img)
    cv2.imshow('crp', buffer[-1])
    cv2.waitKey()

cv2.destroyAllWindows()

Model input shape: [3, 480, 640]
Running with CUDA backend.
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/kornia/geometry/transform/imgwarp.py:98: UserWarning: The align_corners default value has been changed. By default now is set True in order to match cv2.warpPerspective. In case you want to keep your previous behaviour set it to False. This warning will disappear in kornia > v0.6.
  warnings.warn(message)
